### This notebook demonstrates how to evaluate accuracy

In [1]:
from sklearn.neighbors import NearestNeighbors
from TfidfVectorizer import TfidfVectorizer
from tqdm import tqdm
from utils import *

# don't forget to set clean=True to clean the data!
data = load_data_from_json('data/data_with_augmented_examples.json', clean=True)
data_test = load_data_from_json('data/test_200.json')

100%|██████████| 103874/103874 [00:52<00:00, 1969.17it/s]


In [2]:
data_train, data_seen_500, data_unseen_500 = split_seen_unseen(data)
print(len(list(itertools.chain(*data_train.values()))))
print(len(list(itertools.chain(*data_seen_500.values()))))
print(len(list(itertools.chain(*data_unseen_500.values()))))

100%|██████████| 98481/98481 [03:11<00:00, 514.30it/s] 


8689498
500
500


### Fit training data

In [3]:
# fit tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)
X_tfidf = vectorizer.fit_transform(data_train) # <--- fit train data here
words = vectorizer.words

# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

100%|██████████| 129297/129297 [01:54<00:00, 1127.19it/s] 


NearestNeighbors(metric='cosine')

### Test seen data

In [4]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_seen_500.items()): # <--- use seen_500 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 494/494 [03:14<00:00,  2.54it/s]

acc@1: 0.22
acc@10: 0.54
acc@100: 0.82
median rank: 6
standard error of mean rank: 9


(0.22469635627530365,
 0.5364372469635628,
 0.8157894736842105,
 6.0,
 8.84217979734917)

### Test unseen data

In [5]:
y_pred = []
y_gold = []
for word, defi in tqdm(data_unseen_500.items()): # <--- use unseen_500 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 496/496 [05:53<00:00,  1.40it/s]

acc@1: 0.25
acc@10: 0.5
acc@100: 0.79
median rank: 10
standard error of mean rank: 10


(0.25, 0.4959677419354839, 0.7862903225806451, 10.0, 10.094211487496535)

### Test description set (200) data

In [6]:
# fit tfidf vectorizer
vectorizer = TfidfVectorizer(stop_words=None, stemmer=None, norm=False)
X_tfidf = vectorizer.fit_transform(data) # <--- fit all data here
words = vectorizer.words

# fit knn
knn = NearestNeighbors(metric='cosine')
knn.fit(X_tfidf)

y_pred = []
y_gold = []
for word, defi in tqdm(data_test.items()): # <--- use desc_200 data here
    query = list(defi)[0]
    query = vectorizer.transform(query)

    prediction = search(query, knn, words, n=1000)

    y_pred.append(prediction)
    y_gold.append(word)

evaluate(y_pred, y_gold)

100%|██████████| 200/200 [02:13<00:00,  1.50it/s]

acc@1: 0.01
acc@10: 0.02
acc@100: 0.19
median rank: 448
standard error of mean rank: 28


(0.005, 0.02, 0.19, 447.5, 27.65448954509918)